<a href="https://colab.research.google.com/github/phylumcollective/bacteria/blob/main/Compositional-Agents/classification/swarmClassify_YOLO.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Google Drive
The following code mounts your Google drive for use with Google CoLab. If you are not using CoLab, the following code will not work.

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False
    
%cd drive/My Drive/phylum/bacteria/Compositional-Agents/classification/

Mounted at /content/drive
Note: using Google CoLab
/content/drive/My Drive/phylum/bacteria/Compositional-Agents/classification


#import
The following packages will be used to implement our dataset preparation script

In [2]:
import os
import random

# configuration

In [7]:
def prepare_dataset(data_dir, images_dir, weights_folder):
    # create a file for the labelled data (labelled_data.data)
    with open(data_dir + '/' + 'labelled_data.data', 'w') as data:
        # By using '\n' we move to the next line
        data.write('classes = ' + str(c) + '\n')

        # Location of the train.txt file
        data.write('train = ' + data_dir + '/' + 'train.txt' + '\n')

        # Location of the test.txt file
        data.write('test = ' + data_dir + '/' + 'test.txt' + '\n')

        # Location of the classes.names file
        data.write('names = ' + data_dir + '/' + 'classes.names' + '\n')

        # Location where to save weights
        data.write('backup = ' + weights_folder)


    f_val = open(data_dir + "/test.txt", 'w')
    f_train = open(data_dir + "/train.txt", 'w')

    path, dirs, files = next(os.walk(images_dir))
    data_size = int(len(files) / 2)

    ind = 0
    data_test_size = int(0.15 * data_size)  # 15% of files used for testing
    test_array = random.sample(range(data_size), k=data_test_size)

    for f in os.listdir(images_dir):
        if(f.split(".")[1] == "JPG" or "jpg"):
            ind += 1

            if ind in test_array:
                f_val.write(images_dir+'/'+f+'\n')
            else:
                f_train.write(images_dir+'/'+f+'\n')

    print("Dataset prepared. Now ready to train YOLO with your custom images and classes")

In [8]:
# location of dataset
DATA_PATH = "data/swarming_yolo"

# run params
SECTION = 'yolo'
RUN_ID = '0000'
DATA_NAME = 'swarming_yolo'
MODEL_FOLDER = 'models/{}/'.format(SECTION)
MODEL_FOLDER += '_'.join([RUN_ID, DATA_NAME])  # where to save the models
print(MODEL_FOLDER)

if not os.path.exists(MODEL_FOLDER):
    os.makedirs(MODEL_FOLDER)
    os.mkdir(os.path.join(MODEL_FOLDER, 'weights'))  # custom weights folder

data_dir = './data/' + DATA_NAME
images_dir = './data/' + DATA_NAME + '/img/'


models/yolo/0000_swarming_yolo


In [9]:
# create a file classes.names from the classes.txt that the YOLO format uses
# counter for classes
c = 0

with open(data_dir + '/' + 'classes.names', 'w') as names, \
     open(data_dir + '/' + 'classes.txt', 'r') as txt:

    # go through all lines in txt file and writing them into names file
    for line in txt:
        names.write(line)
        # increment counter
        c += 1

In [10]:
# create files labelled_data.data and train.txt and test.txt for train/test split
prepare_dataset(data_dir, images_dir, MODEL_FOLDER + '/weights/')

TypeError: ignored